In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import re
from itertools import chain
import warnings

import spacy
#spacy.cli.download("en_core_web_trf")

import gender_guesser.detector as gender
from allennlp.predictors.predictor import Predictor
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

data_folder = 'data/'

C:\Users\Imane\anaconda3\envs\ada\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
error loading _jsonnet (this is expected on Windows), treating C:\Users\Imane\AppData\Local\Temp\tmptzcxy1bh\config.json as plain json
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Imane\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retra

In [2]:
#Load data and set indexes
movie_metadata = pd.read_csv(data_folder + "movie.metadata.tsv", sep='\t', header = None)
movie_metadata.columns = ['movie_ID','FB_ID','movie_name','release_date','box_office','length','languages','country','genres']

#Force float type to the movie ID column, set as index
movie_metadata.movie_ID = movie_metadata.movie_ID.astype(int)
movie_metadata = movie_metadata.set_index('movie_ID')

#Drop freebase database ID, we don't need it
movie_metadata = movie_metadata.drop(columns=['FB_ID'])

#Remove movies with non-defined release date
#Convert release date to datetime type, only keep year 
movie_metadata = movie_metadata[movie_metadata.release_date.notna()]
movie_metadata['release_date'] = movie_metadata.release_date.apply(lambda d:datetime.strptime(str(d[0:4]), "%Y"))
movie_metadata['release_date'] = [date.year for date in movie_metadata.release_date]

#Change release date 1010, it's a mistake movie actually release in 2010
movie_metadata = movie_metadata.replace(1010,2010)

#Clean languages, country, genres columns
movie_metadata['languages'] = [lang.split('"')[3::4] for lang in movie_metadata.languages]
movie_metadata['country'] = [lang.split('"')[3::4] for lang in movie_metadata.country]
movie_metadata['genres'] = [lang.split('"')[3::4] for lang in movie_metadata.genres]

movie_metadata.head(10)

,movie_name,release_date,box_office,length,languages,country,genres
movie_ID,,,,,,,
975900,Ghosts of Mars,2001,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
28463795,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]"
9363483,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri..."
261236,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama]
13696889,The Gangsters,1913,NaN,35.0,"[Silent film, English Language]",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi..."
18998739,The Sorcerer's Apprentice,2002,NaN,86.0,[English Language],[South Africa],"[Family Film, Fantasy, Adventure, World cinema]"
10408933,Alexander's Ragtime Band,1938,3600000.0,106.0,[English Language],[United States of America],"[Musical, Comedy, Black-and-white]"
9997961,Contigo y aquí,1974,NaN,NaN,[Spanish Language],[Argentina],"[Musical, Drama, Comedy]"


In [3]:
#Load character metadata
character_metadata = pd.read_csv(data_folder + "character.metadata.tsv", sep='\t', header=None)

#only keep movie_ID , name, gender, and age
character_metadata = character_metadata.iloc[:,[0,2,3,5,9]] 
character_metadata.columns = ['movie_ID','release_date','name','gender','age']
character_metadata.name = character_metadata['name'].astype(str)
character_metadata.movie_ID = character_metadata['movie_ID'].astype(int)
character_metadata = character_metadata.set_index('movie_ID')

#Change realease year to float indicading the year
character_metadata = character_metadata[character_metadata['release_date'].notna()]
character_metadata['release_date'] = character_metadata['release_date'].apply(lambda x : datetime.strptime(str(x)[0:4],"%Y"))
character_metadata['release_date'] = [date.year for date in character_metadata['release_date']] 
character_metadata = character_metadata.replace(1010, 2010)

#Drop the characters without gender defined
character_metadata = character_metadata[character_metadata.gender.notna()]

character_metadata.sample(10)

,release_date,name,gender,age
movie_ID,,,,
21966094,1960,Stella,F,NaN
24225314,1931,Red,M,NaN
8172248,2007,Claudine,F,55.0
15310776,1937,nan,F,NaN
5326407,1979,nan,F,22.0
133574,1992,English Bob,M,61.0
31083375,1949,nan,M,32.0
18906393,1959,nan,M,45.0
1065214,1984,Father Vogler,M,31.0


In [4]:
summary_file = open(data_folder + 'plot_summaries.txt', 'r', encoding="utf8")
summaries = summary_file.readlines()

#Separate summary and movie ID
summaries = pd.DataFrame([summ.split("\t") for summ in summaries], columns=['movie_ID','summary'])
summaries['summary'] = summaries.summary.replace(r'\n',' ', regex=True) #Remove /n at end of summaries
summaries['movie_ID'] = summaries['movie_ID'].astype(int)
summaries = summaries.set_index('movie_ID')

#Count number of words in each summary
def count_words_simple(x):
    return len(x.split(" "))

summaries['length'] = summaries['summary'].apply(count_words_simple)

summaries.head()

,summary,length
movie_ID,,
23890098,"Shlykov, a hard-working taxi driver and Lyosha...",26
31186339,The nation of Panem consists of a wealthy Capi...,781
20663735,Poovalli Induchoodan is sentenced for six yea...,505
2231378,"The Lemon Drop Kid , a New York City swindler,...",854
595909,Seventh-day Adventist Church pastor Michael Ch...,398


In [5]:
def check_USA_produced(countries):
    return any('United States of America' in country for country in countries)

#Select only movies produced in the US
mask_usa = [check_USA_produced(x) for x in movie_metadata.country]
usa_movie = movie_metadata[mask_usa][:]

#Selecting only movies released after 1940
usa_movie.drop(usa_movie[usa_movie['release_date'] < 1940.0].index, inplace = True)

usa_movie.head()

,movie_name,release_date,box_office,length,languages,country,genres
movie_ID,,,,,,,
975900,Ghosts of Mars,2001,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
6631279,Little city,1997,NaN,93.0,[English Language],[United States of America],"[Romantic comedy, Ensemble Film, Comedy-drama,..."
77856,Mary Poppins,1964,102272727.0,139.0,[English Language],[United States of America],"[Children's/Family, Musical, Fantasy, Comedy, ..."
33420460,Keep the Change,1992,NaN,97.0,[English Language],[United States of America],[Drama]


In [6]:
#Filter out summaries that are not in the set of movie of interest (USA + released after ...)
movie_interest = list(set(np.unique(summaries.index)).intersection(usa_movie.index.values))
summaries_USA = summaries.loc[movie_interest]

In [7]:
# select summaries having more 350 than 350 words
summaries_more_350words = summaries_USA[(summaries_USA['length'] >= 350)]

In [8]:
# Joining USA movie and summaries metadata
merged_summary = pd.merge(left = summaries_more_350words, right = usa_movie['release_date'], on = 'movie_ID', how = 'right')
#Drop movies with nan
merged_summary = merged_summary.dropna(subset = ['summary'])

In [9]:
from nlp_pipeline import * #just one function ??

gender_guess = gender.Detector(case_sensitive=False)

#List of words to remove from the name of the character 
words_remove = pd.read_csv(data_folder + 'remove_from_name.csv', encoding="utf8", header=None)
words_remove = list(words_remove.iloc[:,0])

#List of interesting words decribing charcaters
family_occupation = pd.read_csv(data_folder + 'family_occupation.csv', encoding="utf8", header=None)
family_occupation = list(family_occupation.iloc[:,0])

In [17]:
remove_idx=(character_metadata[character_metadata.groupby('movie_ID').count().release_date == 1].index.intersection(merged_summary.index))
#Create Serie with movie_id as index and output nlp pipeline 
idx = list(merged_summary.index.intersection(character_metadata.index)) 
#Remove problematic index
idx = [x for x in idx if x not in remove_idx]

idx = idx[100:201]   #CHANGE INDEX BEFORE RUNNING, index + 1 !!!
process_summaries =  pd.DataFrame({'movie_id': idx})

#Apply nlp pipeline 
with warnings.catch_warnings():
    warnings.simplefilter("ignore") #Allenlp use a deprecated version of a function of pytorch, therefore, we supress warnings here.
    process_summaries['nlp'] = process_summaries.movie_id.apply(lambda i: process_summary(summaries.loc[i].summary, character_metadata.loc[i][:], predictor, words_remove, family_occupation, gender_guess))

C:\Users\Imane\AppData\Local\Temp\ipykernel_24520\1098651672.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  remove_idx=(character_metadata[character_metadata.groupby('movie_ID').count().release_date == 1].index.intersection(merged_summary.index))


In [19]:
process_summaries.to_pickle("data/100_200.pkl") #CHANGE NAME before saving, 100_200 (value 200 is included)

In [ ]:
#pd.read_pickle("data/500.pkl").nlp[0]     reads dataframe of first summary